# DATA ANALYSIS OF SECTION 8 HUD FMR DATA

In this notebook, we show how to analyze Section 8 housing fmr dataset stored in IBM dashDB using Apache Spark with Scala .code can be found here https://github.com/RaikiratSohi/Section8_DataAnalysis_SparkScala.git . The FMR dataset contains the rental value that is reimbursed by govt. in different county's around United States.


In [2]:
// Import SQLContext and data types
// sc is an existing SparkContext.

val sqlContext =  new org.apache.spark.sql.SQLContext(sc)

# Data Loading
In this section, we load the data from a .csv file that has been saved in dashDB and automatically create a SparkSQL DataFrame that contains all the data. 

In [3]:
val dashdata = sqlContext.load("jdbc", Map( "url" -> "jdbc:db2://bluemix05.bluforcloud.com:50000/BLUDB:user=dash017994;password=NxNXtoh29gvw;",  "dbtable" -> "dash017994.FMR2015"))
val dashdata2 = sqlContext.load("jdbc", Map( "url" -> "jdbc:db2://bluemix05.bluforcloud.com:50000/BLUDB:user=dash017994;password=NxNXtoh29gvw;",  "dbtable" -> "dash017994.ACTUALHUD"))

# Sending Data to Temp Tables
The fmr tabel data("fmrdata") contains the Fair Market Rents For The Section 8 Housing Assistance Payments Program Data provide by US Department of Housing and Urban Development at http://catalog.data.gov/dataset/fair-market-rents-for-the-section-8-housing-assistance-payments-program . The other database table ("actualdata") is the dummy database that enlists the prevailing rates in different counties at which the Section 8 housing is offered. 

In [4]:
dashdata.registerTempTable("fmrdata")
dashdata2.registerTempTable("actualdata") 

#Print Schema/Layout and Contents of both Dataset tables
dashdata and dashdata2 are tables where data is stored for analysis

In [5]:
dashdata.printSchema
dashdata2.printSchema

root
 |-- FIPS2000: string (nullable = true)
 |-- FIPS2010: string (nullable = true)
 |-- FMR2: integer (nullable = true)
 |-- FMR0: integer (nullable = true)
 |-- FMR1: integer (nullable = true)
 |-- FMR3: integer (nullable = true)
 |-- FMR4: integer (nullable = true)
 |-- COUNTY: integer (nullable = true)
 |-- STATE: integer (nullable = true)
 |-- COUSUB: string (nullable = true)
 |-- COUNTYNAME: string (nullable = true)
 |-- METRO_CODE: string (nullable = true)
 |-- AREANAME: string (nullable = true)
 |-- COUNTY_TOWN_NAME: string (nullable = true)
 |-- POP2010: integer (nullable = true)
 |-- STATE_ALPHA: string (nullable = true)
 |-- FMR_TYPE: integer (nullable = true)
 |-- METRO: integer (nullable = true)



In [6]:
dashdata.show
dashdata2.show

+----------+----------+----+----+----+----+----+------+-----+------+-----------------+----------------+--------------------+--------------------+-------+-----------+--------+-----+
|  FIPS2000|  FIPS2010|FMR2|FMR0|FMR1|FMR3|FMR4|COUNTY|STATE|COUSUB|       COUNTYNAME|      METRO_CODE|            AREANAME|    COUNTY_TOWN_NAME|POP2010|STATE_ALPHA|FMR_TYPE|METRO|
+----------+----------+----+----+----+----+----+------+-----+------+-----------------+----------------+--------------------+--------------------+-------+-----------+--------+-----+
|0901155500|0901155500|1057| 716| 805|1353|1560|    11|    9| 55500|New London County|METRO35980M35980|Norwich-New Londo...|North Stonington ...|   5297|         CT|      40|    1|
|0901156270|0901156270|1057| 716| 805|1353|1560|    11|    9| 56270|New London County|METRO35980M35980|Norwich-New Londo...|        Norwich town|  40493|         CT|      40|    1|
|0901157040|0901157040|1057| 716| 805|1353|1560|    11|    9| 57040|New London County|METRO3598

# Compute the distribution FDR0 such that 
In this section, we demonstrate how to use SparkSQL queries to detect fraud with analysis such that if Actual prevailing Rent value reperesented by ACT_ is greater than 1.4 times of FDR_value then that county need to be checked for fraud since such spike in rent from FDR value can mean prevelant fraud in that county.

For this we will retrieve FMR0 data with each county unique identification no according to 2010 i.e. FIPS2010 
So the sql query stored in result1 multiplies the FMR0 values stored in FMR0 in fmr table and updates it to be saved in results1. 
Then the ACT0 and FIPS2010 from actualdata table are retrieved .
The final join integrates both the tables and output data such that when FIPS2010 in both tables is equal and ACT0 > 1.4 * FMR0 value. The FIPS value with discrepency is printed . 
This can be used to find frauds in different counties.

In [7]:
val result = sqlContext.sql("SELECT FMR0,FIPS2010 from fmrdata")
result.show

val result1 = sqlContext.sql("select (FMR0 * 1.4),FIPS2010 from fmrdata" )
result1.show 

+----+----------+
|FMR0|  FIPS2010|
+----+----------+
| 700|2502771480|
| 674|2502771620|
| 531|2502773090|
| 674|2502773895|
| 674|2502775015|
| 674|2502775155|
| 674|2502775400|
| 610|2502777010|
| 610|2502780405|
| 674|2502782000|
| 448|2600199999|
| 448|2600399999|
| 576|2600599999|
| 411|2600799999|
| 391|2600999999|
| 485|2601199999|
| 448|2601399999|
| 456|2601599999|
| 418|2601799999|
| 549|2601999999|
+----+----------+



In [10]:
val results = sqlContext.sql("SELECT ACT0,FIPS2010 from actualdata")
results.show

+----+---------+
|ACT0| FIPS2010|
+----+---------+
| 788|100199999|
| 762|100399999|
| 670|100599999|
| 773|100799999|
| 773|100999999|
| 599|101199999|
| 599|101399999|
| 675|101599999|
| 696|101799999|
| 599|101999999|
| 599|102199999|
| 708|102399999|
| 599|102599999|
| 599|102799999|
+----+---------+



In [11]:
result1.join(results, result1("c0") < (results("ACT0")) && result1("FIPS2010") === results ("FIPS2010")).show

+-----------------+----------+----+---------+
|               c0|  FIPS2010|ACT0| FIPS2010|
+-----------------+----------+----+---------+
|            763.0|0100999999| 773|100999999|
|691.5999999999999|0100399999| 762|100399999|
|            763.0|0100799999| 773|100799999|
|592.1999999999999|0102199999| 599|102199999|
+-----------------+----------+----+---------+

